### Speaker diarization

In [2]:
import requests
import time
from dotenv import load_dotenv
load_dotenv()
import os
from csv import writer

base_url = "https://api.assemblyai.com"

headers = {
    "authorization": f"{os.getenv("AAI_KEY")}"
}

with open("Transcription/Case 07.m4a", "rb") as f:
  response = requests.post(base_url + "/v2/upload",
                          headers=headers,
                          data=f)

upload_url = response.json()["upload_url"]

data = {
    "audio_url": upload_url, # You can also use a URL to an audio or video file on the web
    "speaker_labels": True
}

url = base_url + "/v2/transcript"
response = requests.post(url, json=data, headers=headers)

transcript_id = response.json()['id']
polling_endpoint = base_url + "/v2/transcript/" + transcript_id

while True:
  transcription_result = requests.get(polling_endpoint, headers=headers).json()

  if transcription_result['status'] == 'completed':
    print(f"Transcript ID:", transcript_id)
    break

  elif transcription_result['status'] == 'error':
    raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

  else:
    time.sleep(3)
transcription = writer(open("transcribe.csv", "w", newline = "", encoding="utf-8"))
transcription.writerow(["speaker", "start", "end", "word"])
for utterance in transcription_result['utterances']:
  print(f"Speaker {utterance['speaker']}: {utterance['text']} from {utterance['start']} to {utterance["end"]}")
  transcription.writerow([utterance["speaker"], utterance["start"], utterance["end"], utterance["text"]])



KeyboardInterrupt: 

### Only transcibing

In [16]:
import requests
import time
from dotenv import load_dotenv
import os
from csv import writer
load_dotenv()

base_url = "https://api.assemblyai.com"

headers = {
    "authorization": os.getenv('AAI_KEY')
}

with open("Transcription/Case 07.m4a", "rb") as f:
  response = requests.post(base_url + "/v2/upload",
                          headers=headers,
                          data=f)

upload_url = response.json()["upload_url"]
print(upload_url)

data = {
    "audio_url": upload_url, # You can also use a URL to an audio or video file on the web
    "speech_model": "universal"
    # "speech_model": "slam-1"
}

url = base_url + "/v2/transcript"
response = requests.post(url, json=data, headers=headers)

transcript_id = response.json()['id']
polling_endpoint = base_url + "/v2/transcript/" + transcript_id

while True:
  transcription_result = requests.get(polling_endpoint, headers=headers).json()

  if transcription_result['status'] == 'completed':
    print(f"Transcript ID:", transcript_id)
    break

  elif transcription_result['status'] == 'error':
    raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

  else:
    time.sleep(3)

csv_writer = writer(open("transcribe.csv", "w", encoding="utf-8", newline=""))

print(transcription_result)
csv_writer.writerow(["word", "start", "end"])
for result in transcription_result["words"]:
    print(f"{result["text"]}, from {result['start']} to {result["end"]}")
    csv_writer.writerow([result["text"], result["start"], result["end"]])


https://cdn.assemblyai.com/upload/31d5df94-403d-4d80-a185-b76e8950cf2b
Transcript ID: 19921fec-e27c-4e0d-8bce-9378b7fd9574
{'id': '19921fec-e27c-4e0d-8bce-9378b7fd9574', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://cdn.assemblyai.com/upload/31d5df94-403d-4d80-a185-b76e8950cf2b', 'text': 'What do you want? B ball. B ball. Ball. Ball, please. Great job.', 'words': [{'text': 'What', 'start': 1120, 'end': 1280, 'confidence': 0.9995117, 'speaker': None}, {'text': 'do', 'start': 1280, 'end': 1480, 'confidence': 1.0, 'speaker': None}, {'text': 'you', 'start': 1480, 'end': 1640, 'confidence': 1.0, 'speaker': None}, {'text': 'want?', 'start': 1640, 'end': 1920, 'confidence': 0.99902344, 'speaker': None}, {'text': 'B', 'start': 2320, 'end': 2720, 'confidence': 0.55810547, 'speaker': None}, {'text': 'ball.', 'start': 4480, 'end': 4880, 'confidence': 0.75146484, 'speaker': None}, {'text': 'B',

In [9]:
for result in transcription_result["words"]:
    print(result)

{'text': 'What', 'start': 1120, 'end': 1280, 'confidence': 0.9995117, 'speaker': None}
{'text': 'do', 'start': 1280, 'end': 1480, 'confidence': 1.0, 'speaker': None}
{'text': 'you', 'start': 1480, 'end': 1640, 'confidence': 1.0, 'speaker': None}
{'text': 'want?', 'start': 1640, 'end': 1920, 'confidence': 0.99902344, 'speaker': None}
{'text': 'B', 'start': 2320, 'end': 2720, 'confidence': 0.55810547, 'speaker': None}
{'text': 'ball.', 'start': 4480, 'end': 4880, 'confidence': 0.75146484, 'speaker': None}
{'text': 'B', 'start': 5680, 'end': 6080, 'confidence': 0.48510742, 'speaker': None}
{'text': 'ball.', 'start': 8560, 'end': 8960, 'confidence': 0.89990234, 'speaker': None}
{'text': 'Ball.', 'start': 9920, 'end': 10320, 'confidence': 0.9970703, 'speaker': None}
{'text': 'Ball,', 'start': 10640, 'end': 11040, 'confidence': 0.95947266, 'speaker': None}
{'text': 'please.', 'start': 11040, 'end': 11440, 'confidence': 0.9995117, 'speaker': None}
{'text': 'Great', 'start': 11440, 'end': 1184

In [10]:
for result in transcription_result["words"]:
    print(f"{result["text"]}, from {result['start']} to {result["end"]}")


What, from 1120 to 1280
do, from 1280 to 1480
you, from 1480 to 1640
want?, from 1640 to 1920
B, from 2320 to 2720
ball., from 4480 to 4880
B, from 5680 to 6080
ball., from 8560 to 8960
Ball., from 9920 to 10320
Ball,, from 10640 to 11040
please., from 11040 to 11440
Great, from 11440 to 11840
job., from 11840 to 12240


In [ ]:
from fastapi import FastAPI

app = FastAPI()


In [13]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from typing import List
import shutil
import os
import uuid # For generating unique filenames

app = FastAPI()

# Directory to save uploaded files
UPLOAD_DIR = "uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

@app.get("/")
async def root():
    return {"message": "Visit /uploadfile or /uploadmultiplefiles to upload files."}

@app.post("/uploadfile/")
async def create_upload_file(
    # Use UploadFile type hint, and File() as the dependency
    # 'file' here will correspond to the field name in the form data
    file: UploadFile = File(...)
):
    """
    Uploads a single file.
    """
    if not file.filename:
        raise HTTPException(status_code=400, detail="No file selected")

    # Securely generate a unique filename
    # You might want to sanitize file.filename further in a real app
    # For now, we'll just append a UUID to prevent collisions
    file_extension = os.path.splitext(file.filename)[1]
    unique_filename = f"{uuid.uuid4()}{file_extension}"
    file_path = os.path.join(UPLOAD_DIR, unique_filename)

    try:
        # Open the file in binary write mode
        # Use shutil.copyfileobj for efficient streaming
        with open(file_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
        
        return {
            "filename": file.filename,
            "content_type": file.content_type,
            "file_size": file.size, # Note: file.size might be None until read in some cases
            "saved_as": unique_filename,
            "message": "File uploaded successfully!"
        }
    except Exception as e:
        # Log the error for debugging
        print(f"Error uploading file: {e}")
        raise HTTPException(status_code=500, detail=f"Could not upload file: {e}")
    finally:
        # Ensure the UploadFile object is closed
        # This is usually handled by FastAPI, but good practice if you manually
        # consume its content or pass it around.
        file.file.close()


@app.post("/uploadmultiplefiles/")
async def create_upload_files(
    # For multiple files, use List[UploadFile]
    files: List[UploadFile] = File(...)
):
    """
    Uploads multiple files.
    """
    uploaded_info = []
    for file in files:
        if not file.filename:
            # This case shouldn't typically happen if the client sends a proper multi-part form
            continue # Skip empty file entries

        file_extension = os.path.splitext(file.filename)[1]
        unique_filename = f"{uuid.uuid4()}{file_extension}"
        file_path = os.path.join(UPLOAD_DIR, unique_filename)

        try:
            with open(file_path, "wb") as buffer:
                shutil.copyfileobj(file.file, buffer)
            
            uploaded_info.append({
                "filename": file.filename,
                "content_type": file.content_type,
                "file_size": file.size,
                "saved_as": unique_filename,
                "status": "success"
            })
        except Exception as e:
            uploaded_info.append({
                "filename": file.filename,
                "status": "failed",
                "error": str(e)
            })
        finally:
            file.file.close()
            
    if not uploaded_info:
        raise HTTPException(status_code=400, detail="No files were uploaded or selected.")

    return {"uploaded_files": uploaded_info}


# To run this API:
# uvicorn main:app --reload

In [14]:
! uvicorn .:app --reload

^C
